In [1]:
%%writefile kinect.py

import pyrealsense2 as rs
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image  # 이미지를 다루기 위한 라이브러리
import time

# 카메라로 인식한 워크스페이스를 아래 범위에 따라 자름. 
WS_PC = [180, 360, 300, 620]  # -> [y축 시작, y축 끝, x축 시작, x축 끝]

# RealSense 카메라를 사용하여 이미지를 수집하고, 수집된 이미지를 실시간으로 시각화
def get_workspace_crop(img):
    retval = img[WS_PC[0]:WS_PC[1], WS_PC[2]:WS_PC[3], ...]
    return retval


class RealSenseClient:
    def __init__(self):
        self.pipeline = rs.pipeline()
        self.config = rs.config()
        self.config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
        self.config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
        self.pipeline.start(self.config) # 파이프라인 시작

    def get_camera_data(self):
        # 프레임 기다렸다가 컬러, 깊이 프레임 가져오기
        frames = self.pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()
        depth_frame = frames.get_depth_frame()

        # 프레임 못 가져오면 color_image, depth_image 모두 None으로 반환
        if not color_frame or not depth_frame:
            return None, None 

        # 받아온 이미지를 처리할 수 있도록 array로 변환
        color_image = np.asanyarray(color_frame.get_data())
        depth_image = np.asanyarray(depth_frame.get_data())

        # depth image 배열을 meter 단위로 스케일 조정
        depth_image = depth_image * 0.001  

        return color_image, depth_image

    def stop(self):
        self.pipeline.stop()


if __name__ == '__main__':
    realsense = RealSenseClient()
    
    counter = 0
    limit = 20
    sleep = 0.05
    reset_interval = 16  # 스트림을 재설정하는 간격

    all_rgbs = []
    try:
        while counter < limit:
            # counter가 reset_interval의 배수이거나, 0이 아닐 때 재설정 -> RealSense 카메라 스트림을 중지하고 다시 시작하는 과정
            if counter % reset_interval == 0 and counter != 0:
                realsense.stop()
                time.sleep(1)  # 스트림이 완전히 종료되도록 잠시 대기
                realsense = RealSenseClient()
            
            color_im, depth_im = realsense.get_camera_data()
            if color_im is None or depth_im is None:
                continue

            # numpy 배열 형태의 이미지를 PIL 이미지 객체로 변환 -> 이미지 파일로 저장, 이미지를 화면에 표시 
            # im = Image.fromarray(img)
            
            print("img shape: ", color_im.shape)
            print("depth shape: ", depth_im.shape)
            counter += 1
            time.sleep(sleep)
            print('Step counter at {}'.format(counter))
            all_rgbs.append(color_im)

            fig, ax = plt.subplots(2, 2, figsize=(10, 5))
        
            plt.show()
    finally:
        realsense.stop()

Overwriting kinect.py


In [5]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.backends.cudnn.version())
print(torch.cuda.is_available())

1.12.0+cu102
10.2
7605
True


In [4]:
print(torch.backends.cudnn.version())


7605
